#### Dockerfile creation
we first create a Dockerfile based on the documentation here https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-jl-admin-guide-custom-images.html#studio-updated-jl-custom-images-dockerfile-templates

```
FROM public.ecr.aws/amazonlinux/amazonlinux:2023

ARG NB_USER="sagemaker-user"
ARG NB_UID=1000
ARG NB_GID=100

# Install Python3, pip, sudo, and other dependencies
RUN dnf install -y \
    python3 \
    python3-pip \
    python3-devel \
    gcc \
    shadow-utils \
    sudo && \
    useradd --create-home --shell /bin/bash --gid "${NB_GID}" --uid ${NB_UID} ${NB_USER} && \
    echo "${NB_USER} ALL=(ALL) NOPASSWD:ALL" > /etc/sudoers.d/${NB_USER} && \
    chmod 0440 /etc/sudoers.d/${NB_USER} && \
    dnf update -y kernel && \
    dnf clean all

RUN python3 -m pip install --no-cache-dir \
    'jupyterlab>=4.0.0,<5.0.0' \
    urllib3 \
    jupyter-activity-monitor-extension \
    --ignore-installed

# Verify versions
RUN python3 --version && \
    jupyter lab --version

USER ${NB_UID}
CMD jupyter lab --ip 0.0.0.0 --port 8888 \
    --ServerApp.base_url="/jupyterlab/default" \
    --ServerApp.token='' \
    --ServerApp.allow_origin='*'
```

In [18]:
# we create the ECR repository
!aws ecr create-repository --repository-name smstudio-custom-sudo-nkernel --region us-east-1

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:609009159737:repository/smstudio-custom-sudo-nkernel",
        "registryId": "609009159737",
        "repositoryName": "smstudio-custom-sudo-nkernel",
        "repositoryUri": "609009159737.dkr.ecr.us-east-1.amazonaws.com/smstudio-custom-sudo-nkernel",
        "createdAt": 1753313603.184,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [19]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 609009159737.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [3]:
!pip install sagemaker-studio-image-build

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'sagemaker-studio-image-build' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sagemaker-studio-image-build'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13558 [REDACTED]54f14b34252b7d71d83c1cfb232b810
  Stored in directory: /home/ec2-user/.[REDACTED]2c9be00f206badea11b5148626ef58c0e0dc
Successfully built sagemaker-studio-image-build


In [20]:
!aws sts get-caller-identity

{
    "UserId": "[REDACTED]U:SageMaker",
    "Account": "609009159737",
    "Arn": "arn:aws:sts::609009159737:assumed-role/SageMaker-ExecutionRole-20250723T112596/SageMaker"
}


make sure to give the proper permissinos and trust policy. An example trust policy is 
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AssumeRole",
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "sagemaker.amazonaws.com",
                    "codebuild.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

In [21]:
!sm-docker build -t smstudio-custom-sudo-nkernel:my-jupyterlab-image .

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-[REDACTED]ions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Created ECR repository sagemaker-studio
....................[Container] 2025/07/23 23:41:50.466313 Running on CodeBuild On-demand

[Container] 2025/07/23 23:41:50.466323 Waiting for agent ping
[Container] 2025/07/23 23:41:52.679737 Waiting for DOWNLOAD_SOURCE
[Container] 2025/07/23 23:41:52.996793 Phase is DOWNLOAD_SOURCE
[Container] 2025/07/23 23:41:52.997965 CODEBUILD_SRC_DIR=/codebuild/output/src2562656751/src
[Container] 2025/07/23 23:41:52.998818 YAML location is [REDACTED]dspec.yml
[Container] 2025/07/23 23:41:53.002382 Setting HTTP c

In [8]:
# 609009159737.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio:latest

# Create an AppImageConfig:

```
{
  "AppImageConfigName": "custom-image-sudo-nkernel-config",
    "JupyterLabAppImageConfig": { 
        "FileSystemConfig": { 
            "MountPath": "/home/sagemaker-user",
            "DefaultUid": 1000,
            "DefaultGid": 100
      }
   }
}
```

In [23]:
!aws sagemaker create-app-image-config --cli-input-json file://app-image-config-input.json --region us-east-1

{
    "AppImageConfigArn": "arn:aws:sagemaker:us-east-1:609009159737:app-image-config/custom-image-sudo-nkernel-config2"
}


In [25]:
# arn:aws:sagemaker:us-east-1:609009159737:app-image-config/custom-image-sudo-nkernel-config2

In [26]:
!aws sagemaker create-image \
  --image-name custom-image-sudo-nkernel2 \
  --role-arn arn:aws:iam::609009159737:role/service-role/SageMaker-ExecutionRole-20250723T112596 \
  --region us-east-1

{
    "ImageArn": "arn:aws:sagemaker:us-east-1:609009159737:image/custom-image-sudo-nkernel2"
}


In [27]:
# arn:aws:sagemaker:us-east-1:609009159737:image/custom-image-sudo-nkernel2

In [28]:
!aws sagemaker create-image-version \
  --image-name custom-image-sudo-nkernel2 \
  --base-image 609009159737.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio:latest \
  --region us-east-1

{
    "ImageVersionArn": "arn:aws:sagemaker:us-east-1:609009159737:image-version/custom-image-sudo-nkernel2/1"
}


In [29]:
# arn:aws:sagemaker:us-east-1:609009159737:image-version/custom-image-sudo-nkernel/1

In [30]:
!aws sagemaker update-domain \
  --domain-id d-hbzcqghviqax \
  --default-space-settings file://default-space-settings.json \
  --region us-east-1

{
    "DomainArn": "arn:aws:sagemaker:us-east-1:609009159737:domain/d-hbzcqghviqax"
}


In [31]:
!aws sagemaker describe-domain --domain-id d-hbzcqghviqax

{
    "DomainArn": "arn:aws:sagemaker:us-east-1:609009159737:domain/d-hbzcqghviqax",
    "DomainId": "d-hbzcqghviqax",
    "DomainName": "lustre-domain",
    "HomeEfsFileSystemId": "fs-0005be7f0673b1850",
    "[REDACTED]": "ins-66cbfc7e5e43d657",
    "SingleSignOnApplicationArn": "arn:aws:sso::609009159737:application/ssoins-7223fd66a8e838ce/apl-66cbfc7e5e43d657",
    "Status": "InService",
    "CreationTime": 1753291559.228,
    "LastModifiedTime": 1753314520.579,
    "AuthMode": "SSO",
    "DefaultUserSettings": {
        "ExecutionRole": "arn:aws:iam::609009159737:role/service-role/SageMaker-ExecutionRole-20250723T112596",
        "SecurityGroups": [],
        "SharingSettings": {
            "NotebookOutputOption": "Allowed",
            "S3OutputPath": "s3://sagemaker-studio-tkpi4a0pt5o/sharing"
        },
        "JupyterServerAppSettings": {
            "DefaultResourceSpec": {
                "SageMakerImageArn": "arn:aws:sagemaker:us-east-1:081325390199:image/jupyter-server-3"